<a href="https://colab.research.google.com/github/Murcicrum/Scrapers-de-inmobiliarias/blob/main/scraper_properati.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####Importo módulos

In [ ]:
from numpy import random
import pandas as pd
import json
import time
import requests
from bs4 import BeautifulSoup as bs


#####Defino la url de properati y los strings que defines mi busqueda

In [ ]:
#URL de Properati con deptos en alquiler en caba, con menos de 5 años de antigüedad
url_properati = 'https://www.properati.com.ar'

#String que define las busquedas: deptos, casas y phs en caba, ordenadas por fecha de publicación
#El número de pagina queda a completar, cada página muestra hasta 30 publicaciones
string_search_deptos = '/s/capital-federal/departamento/alquiler?sort=published_on_desc&page='
string_search_casas = '/s/capital-federal/casa/alquiler?sort=published_on_desc&page='
string_search_phs = '/s/capital-federal/ph/alquiler?page='

#por si pinta 403
#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

#####Extraigo todos los links a las publicaciones que devuelve

In [ ]:
def get_links(string_search:str, N_pags:int) -> list:
  
  url_properati='https://www.properati.com.ar'
  url_deptos = []
  
  for i in range(1,N_pags+1):
    url_search = url_properati + string_search + str(i)
    response = requests.get( url_search )

    if response.status_code==200:
      print('Extrayendo links de página', i)
      soup = bs(response.content)
      tag_deptos = soup.findAll(name='div', attrs={'class':'StyledCardInfo-n9541a-2'})

      for j,t in enumerate(tag_deptos):
        url_depto = url_properati + t.find(name='a').attrs['href']
        if url_depto not in url_deptos:
          url_deptos.append( url_depto )
    
    else:
      print(f'Fallo. Página: {i}, Status_code: {response.status_code}, URL: {url}')
    
    time.sleep(5*random.rand())  
  print(f'Se extrajeron {len(url_deptos)} links')

  return url_deptos

In [ ]:
url_deptos = get_links( string_search_deptos, 10)

Extrayendo links de página 1
Extrayendo links de página 2
Extrayendo links de página 3
Extrayendo links de página 4
Extrayendo links de página 5
Extrayendo links de página 6
Extrayendo links de página 7
Extrayendo links de página 8
Extrayendo links de página 9
Extrayendo links de página 10
Se extrajeron 232 links


#####Para cada publicación extraer la data que nos interesa

In [ ]:
import csv

def find_value(keys: str, dic: dict):
  '''
  Dado una secuencia de keys anidadas en el dict
  Si encuentra cada una de las keys devuelve el valor, buscando recursivamente
  '''
  list_keys = keys.split('.')
  if list_keys[0] in dic:

    if len(list_keys)==1:
      return dic[ list_keys[0] ]
    else:
      new_keys = '.'.join(list_keys[1:])
      return find_value( new_keys, dic[list_keys[0]] )
  
  else:
    return None


def get_data(urls: list,  to_extract: dict, filename: str='', n: int=30) -> list:
  '''
  Dada una lista con urls de publicaciones de properati
  Arma una lista de diccionarios, donde cada uno tiene la misma estructura:
    keys = las keys de to_extract
    values = el valor que tenga el json de cada publicación en la secuencia de keys definida en los values de to_extract.
  Si el json de una publicación no contiene una secuencia de keys expresada en un value de to_extract,
  el valor que toma el diccionario es None.
  Si se encuentras publicaciones finalizadas en urls o falló el request, se omiten.
  ###REESCRBIR MEJOR
  '''

  data_list = []      
    
  for i, url in enumerate(urls):
#Chequeo si el request sale bien y la publicación está disponible    
    response = requests.get(url)
    if response.status_code != 200:
      print(i, 'Falló en link \t',  'Status_code: ', response.status_code,'\t' ,url)
      continue

    soup = bs(response.content)

    if 'Esta publicación ya no está disponible' == soup.find(name='h1', attrs={'class':'StyledTitle-mffpuc-0'}).text:
      print(i, 'Finalizó la publicación ', url)
      continue

#Levanto los datos y los guardo en una lista
    data_json = json.loads( soup.find(name='script', attrs={'id':'__NEXT_DATA__'}).string )
    d_props = data_json['props']['pageProps']['property']

    data = {}

    for r, s in to_extract.items():
      data[r] = find_value( s, d_props )
    data['url']=url
   
    data_list.append(data)
    
#####Guardo los datos de a lotes si me diste un path
    if filename and (i%n==0 or i==(len(urls)-1)):     
      with open(filename, 'a') as f:  # You will need 'wb' mode in Python 2.x
        w = csv.DictWriter(f, fieldnames=data.keys())
        if i==0:  
          print('Los datos se guardarán en', filename)
          w.writeheader()
          w.writerow( data_list[0] )
        else:
          print(i, 'Guardando...')
          if i%n==0:
            for data in data_list[-n:]:
              w.writerow(data)
          elif i==len(urls)-1:
            for data in data_list[-(i%n):]:
              w.writerow(data)

    time.sleep(5*random.rand())  

  return data_list

#####Defino la data que quiero

In [ ]:
#cada key es la key con la que quedará registrada cada valor que querramos extraer del json
#cada value es un string con la secuencia de keys a recorrer en el json para extraer el valor deseado
to_extract = {'ub_calle': 'address.street',                                     #
              'ub_lat': 'geo_point.lat',                                        #
              'ub_lon': 'geo_point.lon',                                        #
              'pr_moneda': 'price.currency',                                    #
              'pr_valor': 'price.amount',                                       #
              'nu_ambs': 'floor_plan.rooms',                                    #
              'nu_habs': 'floor_plan.bedrooms',                                 #
              'sp_des': 'surface.total',                                        #
              'sp_cub': 'surface.covered',                                      #
              'fe_pub': 'published_on'}                                         #

#####Descargo la data y la guardo


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

#os.listdir('drive/MyDrive/Colab Notebooks/Espacios públicos TDP/')
#os.remove(save_filename)
#os.listdir('drive/MyDrive/Colab Notebooks/Espacios públicos TDP/')
path_drive = 'drive/MyDrive/Colab Notebooks/Espacios públicos TDP/'

save_filename = path_drive + 'properati_deptos.csv'

n_guardar = 10                #Cada cuantas entradas guardo en el csv
data_deptos = get_data(url_deptos, to_extract, save_filename, n_guardar  )

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Los datos se guardarán en drive/MyDrive/Colab Notebooks/Espacios públicos TDP/properati.csv
30 Guardando...
60 Guardando...
68 Finalizó la publicación  https://www.properati.com.ar/detalle/3ynrq_alquiler_departamento_colegiales_subte-linea-d_ciclovias-y-bicisendas-de-buenos-aires_cocina_encargado_vigilancia_gladys-fontela-soluciones-inmobiliarias_hmbr
90 Guardando...
120 Guardando...
150 Guardando...
180 Guardando...
210 Guardando...
231 Guardando...


Y ahora los puedo cargar en un dataframe

In [ ]:
df = pd.read_csv(save_filename)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ub_calle   357 non-null    object
 1   ub_lat     357 non-null    object
 2   ub_lon     357 non-null    object
 3   pr_moneda  358 non-null    object
 4   pr_valor   358 non-null    object
 5   nu_ambs    72 non-null     object
 6   nu_habs    257 non-null    object
 7   sp_des     331 non-null    object
 8   sp_cub     50 non-null     object
 9   fe_pub     358 non-null    object
 10  url        358 non-null    object
dtypes: object(11)
memory usage: 30.9+ KB


#Repito lo mismo para casas y phs

In [ ]:
url_casas = get_links( string_search_deptos, 10)
save_filename = path_drive + 'properati_casas.csv'
data_casas = get_data( url_casas, to_extract, save_filename, n_guardar )

url_phs = get_links( string_search_phs, 10)
save_filename = path_drive + 'properati_phs.csv'
data_phs = get_data( url_phs, to_extract, save_filename, n_guardar )